In [3]:
import concurrent, glob, multiprocessing, os, subprocess, sys, threading, time, traceback, urllib2

def download_file(url, filename):
    if os.path.exists(filename):
        sys.stdout.write('%s already downloaded\n' % filename)
    else:
        if not os.path.exists(os.path.dirname(filename)):
            os.makedirs(os.path.dirname(filename))
        sys.stdout.write('Downloading %s to %s\n' % (url, filename))
        data = urllib2.urlopen(url).read()
        open(filename + '.tmp', "wb").write(data)
        os.rename(filename + '.tmp', filename)
        sys.stdout.write('Done, wrote %d bytes to %s\n' % (len(data), filename))
        
def subprocess_check(*args, **kwargs):
    if len(args) == 1 and type(args[0]) == str:
        kwargs['shell'] = True
        cmd = args
    else:
        cmd = ' '.join(args)
    p = subprocess.Popen(*args,  
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         **kwargs)
    (out, err) = p.communicate()
    ret = p.wait()
    if ret != 0:
        if out and out[-1] != '\n':
            out += '\n'
        if err and err[-1] != '\n':
            err += '\n'
        raise Exception(
            ('Call to subprocess_check failed with return code {ret}\n'
             'Command: {cmd}\n'
             'Standard error:\n{err}\n'
             'Standard out:\n{out}\n').format(**locals()))
    if err != '' and out != '' and err[-1] != '\n':
        err += '\n'
    return err + out

class Stopwatch:
    def __init__(self, msg):
        self.msg = msg
        
    def __enter__(self):
        self.start = time.time()
        
    def __exit__(self, a, b, c):
        print '%s took %.1f seconds' % (self.msg, time.time() - self.start)
        
class Promise:
    def __init__(self, *args):
        def thunk():
            self.result = apply(args[0], args[1:])
        self.thread = threading.Thread(target=thunk)
        self.thread.start()
        
    def value(self):
        self.thread.join()
        return self.result
    
class SimpleThreadPoolExecutor(concurrent.futures.ThreadPoolExecutor):
    def __init__(self, max_workers):
        super(SimpleThreadPoolExecutor, self).__init__(max_workers=max_workers)
        self.futures = []
        
    def submit(self, fn, *args, **kwargs):
        future = super(SimpleThreadPoolExecutor, self).submit(fn, *args, **kwargs)
        self.futures.append(future)
        return future
    
    def get_futures(self):
        return self.futures

    def shutdown(self):
        exception_count = 0
        results = []
        for completed in concurrent.futures.as_completed(self.futures):
            try:
                results.append(completed.result())
            except Exception as e:
                exception_count += 1
                sys.stderr.write(
                    'Exception caught in SimpleThreadPoolExecutor.shutdown.  Continuing until all are finished.\n' +
                    'Exception follows:\n' +
                    traceback.format_exc())
        super(SimpleThreadPoolExecutor, self).shutdown()
        if exception_count:
            raise Exception('SimpleThreadPoolExecutor failed: %d of %d raised exception' % (exception_count, len(self.futures)))
        print 'SimpleThreadPoolExecutor succeeded: all %d jobs completed' % (len(self.futures))
        return results

max_subprocess_parallelism = 8
subprocess_semaphore = multiprocessing.Semaphore(max_subprocess_parallelism)
def throttled_subprocess_check(cmd):
    subprocess_semaphore.acquire()
    try:
        subprocess_check(cmd)
    finally:
        subprocess_semaphore.release()    

max_download_parallelism = 6
download_semaphore = multiprocessing.Semaphore(max_download_parallelism)
def throttled_download_file(src, dest):
    with download_semaphore:
        download_file(src, dest)

        
        

## Download sections and build VRTs

In [22]:
webmerc_halfwidth = 20037508.342789244
webmerc_width = webmerc_halfwidth * 2

def correct_and_reproject(src, row, col):
    dest = os.path.splitext(src)[0] + '_corrected.tif'
    if os.path.exists(dest):
        print '{dest} already exists'.format(**locals())
    else:
        tmp = dest + '.tmp.tif'
        n = 90 - 90 * row
        s = n - 90
        w = -180 + 90 * col
        e = w + 90
        try:
            os.unlink(tmp)
        except:
            pass
        cmd = 'gdal_translate -a_srs EPSG:4326 -a_ullr {w} {n} {e} {s} {src} {tmp}'
        cmd = cmd.format(**locals())
        print cmd
        throttled_subprocess_check(cmd)
        os.rename(tmp, dest)
    
    for subrow in range(row*2, row*2+2):
        n = - webmerc_width * (subrow + 1) / 4.0 + webmerc_halfwidth
        s = - webmerc_width * (subrow + 0) / 4.0 + webmerc_halfwidth
        w = webmerc_width * col / 4.0 - webmerc_halfwidth
        e = webmerc_width * (col + 1) / 4.0 - webmerc_halfwidth
        proj_dest = os.path.splitext(dest)[0] + '_projected_' + 'NS'[subrow % 2] + '.tif'
        if os.path.exists(proj_dest):
            print '{proj_dest} already exists'.format(**locals())
        else:
            proj_tmp = proj_dest + '.tmp.tif'
            cmd = 'gdalwarp {dest} -t_srs EPSG:3857'
            cmd += ' -r lanczos'
            cmd += ' -te {w} {n} {e} {s}'
            cmd += ' -ts 32768 32768'
            cmd += ' {proj_tmp}'
            cmd = cmd.format(**locals())
            try:
                os.unlink(proj_tmp)
            except:
                pass
            print cmd
            throttled_subprocess_check(cmd)
            os.rename(proj_tmp, proj_dest)
        
    print 'Corrected projection on {src} and reprojected to web mercator'.format(**locals())
 
def download_correct_and_reproject(src, dest, row, col):
    throttled_download_file(src, dest)
    correct_and_reproject(dest, row, col)

pool = SimpleThreadPoolExecutor(30)

for year in [2012, 2016]:
    !mkdir -p $year
    for row in range(2):
        rowname = '12'[row]
        for col in range(4):
            colname = 'ABCD'[col]
            filename = 'BlackMarble_{year}_{colname}{rowname}_geo.tif'.format(**locals())
            src = 'https://www.nasa.gov/specials/blackmarble/{year}/tiles/georeferrenced/'.format(**locals()) + filename
            dest = '%d/%s' % (year, filename)
            pool.submit(download_correct_and_reproject, src, dest, row, col)

pool.shutdown()

2012/BlackMarble_2012_A1_geo.tif already downloaded
2012/BlackMarble_2012_A1_geo_corrected.tif already exists
2012/BlackMarble_2012_A1_geo_corrected_projected_N.tif already exists
2012/BlackMarble_2012_B1_geo.tif already downloaded
2012/BlackMarble_2012_A1_geo_corrected_projected_S.tif already exists
Corrected projection on 2012/BlackMarble_2012_A1_geo.tif and reprojected to web mercator2012/BlackMarble_2012_C1_geo.tif already downloaded

2012/BlackMarble_2012_B1_geo_corrected.tif already exists
2012/BlackMarble_2012_D1_geo.tif already downloaded
2012/BlackMarble_2012_D1_geo_corrected.tif already exists
2012/BlackMarble_2012_D1_geo_corrected_projected_N.tif already exists
2012/BlackMarble_2012_C1_geo_corrected.tif already exists
 2012/BlackMarble_2012_B1_geo_corrected_projected_N.tif already exists2012/BlackMarble_2012_D1_geo_corrected_projected_S.tif already exists2012/BlackMarble_2012_C1_geo_corrected_projected_N.tif already exists


2012/BlackMarble_2012_A2_geo.tif already downloade

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [23]:
!gdalbuildvrt 2012.vrt 2012/*corrected_projected_?.tif
!gdalinfo 2012.vrt | grep 'Size is'

0...10...20...30...40...50...60...70...80...90...100 - done.
Size is 131072, 131072


In [15]:
!gdalinfo 2012/BlackMarble_2012_A2_geo_corrected_projected_N.tif

Driver: GTiff/GeoTIFF
Files: 2012/BlackMarble_2012_A2_geo_corrected_projected_N.tif
Size is 32768, 32768
Coordinate System is:
PROJCS["WGS 84 / Pseudo-Mercator",
    GEOGCS["WGS 84",
        DATUM["WGS_1984",
            SPHEROID["WGS 84",6378137,298.257223563,
                AUTHORITY["EPSG","7030"]],
            AUTHORITY["EPSG","6326"]],
        PRIMEM["Greenwich",0],
        UNIT["degree",0.0174532925199433],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Mercator_1SP"],
    PARAMETER["central_meridian",0],
    PARAMETER["scale_factor",1],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0],
    UNIT["metre",1,
        AUTHORITY["EPSG","9001"]],
    EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext  +no_defs"],
    AUTHORITY["EPSG","3857"]]
Origin = (-20037508.342799998819828,10018754.171399999409914)
Pixel Size = (305.748113140869123,-305.748113140869123)
Metadata:
  AREA_OR_POINT

In [ ]:
gdalwarp {src} -t_srs EPSG:3785 {row_tmp} -r lanczos

In [27]:
year = 2012




#nlevels = 9


def compute_row(src, dest, level, row):
    row_filename = '{dest}/{level}/{row}/row.tif'.format(**locals())
    try:
        os.makedirs(os.path.dirname(row_filename))
    except:
        pass
    if os.path.exists(row_filename):
        return row_filename
    row_tmp = row_filename + '.tmp.tif'
    try:
        os.unlink(row_tmp)
    except:
        pass
    if level == max_level:
        n = - webmerc_width * (row + 1.0) / 2**level + webmerc_halfwidth
        s = - webmerc_width * (row + 0.0) / 2**level + webmerc_halfwidth
        e = webmerc_halfwidth
        w = -webmerc_halfwidth
        width_px = 256 * 2 ** level
        height_px = 256
        cmd = 'gdalwarp {src} -t_srs EPSG:3785 {row_tmp}'
        # cmd += ' -r lanczos'
        cmd += ' -te {w} {n} {e} {s}'
        cmd += ' -ts {width_px} {height_px}'
        cmd = cmd.format(**locals())
        #print cmd
        throttled_subprocess_check(cmd)
    else:
        row1_promise = Promise(compute_row, src, dest, level + 1, row * 2)        
        row2 = compute_row(src, dest, level + 1, row * 2 + 1)
        row1 = row1_promise.value()
        montage_tmp = row_tmp + '.montage.tmp'
        cmd = 'montage -geometry x256 -tile 1 {row1} {row2} {montage_tmp}'.format(**locals())
        #print cmd
        throttled_subprocess_check(cmd)
        cmd = 'convert -resize 50% {montage_tmp} {row_tmp}'.format(**locals())
        #print cmd
        throttled_subprocess_check(cmd)
        os.unlink(montage_tmp)
        os.unlink(row1)
        os.unlink(row2)
    sys.stdout.write('{row_filename} created\n'.format(**locals()))
    
    row_dir = os.path.dirname(row_filename)
    cmd = 'convert {row_tmp} -crop 256x256 -set filename:tile "%[fx:page.x/256]"'
    cmd += ' +repage +adjoin "{row_dir}/%[filename:tile].jpg"'
    cmd = cmd.format(**locals())
    #print cmd
    throttled_subprocess_check(cmd)
    os.rename(row_tmp, row_filename)
    sys.stdout.write('{row_dir} tiles created\n'.format(**locals()))
    return row_filename

#src = '2012small.tif'
#dest = '2012small.tiles'

src = '2012.vrt'
dest = '2012.tiles'

max_level = 5
maxdim = 256 * 2**max_level
print 'Building tile levels 0-{max_level}, {maxdim}px x {maxdim}px'.format(**locals())

!rm -f 2012.tiles/*/*/row.tif
with Stopwatch('Create all tiles'):
    compute_row(src, dest, 0, 0)

Building tile levels 0-5, 8192px x 8192px
2012.tiles/5/29/row.tif created
2012.tiles/5/31/row.tif created
2012.tiles/5/27/row.tif created
2012.tiles/5/21/row.tif created
2012.tiles/5/5/row.tif created
2012.tiles/5/7/row.tif created
2012.tiles/5/23/row.tif created
2012.tiles/5/19/row.tif created
2012.tiles/5/15/row.tif created
2012.tiles/5/25/row.tif created
2012.tiles/5/26/row.tif created
2012.tiles/5/22/row.tif created
2012.tiles/5/3/row.tif created
2012.tiles/5/17/row.tif created
2012.tiles/5/18/row.tif created
2012.tiles/5/6/row.tif created
2012.tiles/5/11/row.tif created
2012.tiles/5/24/row.tif created
2012.tiles/5/4/row.tif created
2012.tiles/5/28/row.tif created
2012.tiles/5/20/row.tif created
2012.tiles/5/2/row.tif created
2012.tiles/5/30/row.tif created
2012.tiles/5/14/row.tif created
2012.tiles/5/13/row.tif created
2012.tiles/5/29 tiles created
2012.tiles/5/31 tiles created
2012.tiles/5/27 tiles created
2012.tiles/5/21 tiles created
2012.tiles/5/5 tiles created
2012.tiles/5/7 

In [26]:
!rm -rf 2012.tiles

In [ ]:
# to level 5:
# 25.4 seconds

# to level 4:
# 36.5 secs w/o threads
# 11.5 sec w 6 threads
# 10.6 sec with 8 threads



## To do:

Figure out Web Mercator bounds in lat, lon
Figure out half lat Web Mercator bounds (ugh)
Try gdal_warp the 8 lights tiles to 16 web mercator, correct resolution (32768x32768)

In [ ]:
131072/4

In [ ]:
!rm 2012small.tif
!gdalwarp -ts 8192 4096 2012.vrt 2012small.tif

In [ ]:
!rm -f test.tif
!gdalwarp 2012.vrt -r cubic -t_srs EPSG:3785 test.tif -te -20037508.3428 16280475.5285 20037508.3428 17532819.7999 -ts 8192 256

In [ ]:
!gdalwarp --help

In [ ]:
20037508.3428/2

In [ ]:
!rm -f zz.tif


In [ ]:
!ls -l zz.tif

In [ ]:
!gdalinfo zz.tif

In [ ]:
gdalwarp 2012.vrt -t_srs EPSG:3857 2012.tiles/5/7/row.tif.tmp.tif -r lanczos -te -20037508.3428 10018754.1714 20037508.3428 11271098.4428 -ts 8192 256

In [ ]:
!gdalinfo zz.tif

In [ ]:
!ls 2012/*corrected.tif

In [ ]:
!gdal_translate --help